In [1]:
!pip install -U "transformers>=4.42.3" bitsandbytes accelerate peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 807.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 81.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [2]:
import os
import copy
from dataclasses import dataclass

import numpy as np
import pandas as pd
import functools
import torch
import torch.nn.functional as F
from datasets import Dataset, DatasetDict
from transformers import (
    BitsAndBytesConfig,
    Gemma2ForSequenceClassification,
    GemmaTokenizerFast,
    Gemma2Config,
    PreTrainedTokenizerBase, 
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from sklearn.metrics import log_loss, accuracy_score, f1_score
from skmultilearn.model_selection import iterative_train_test_split

In [3]:
@dataclass
class Config:
    output_dir: str = 'output'
    checkpoint: str = 'unsloth/gemma-2-9b-it-bnb-4bit'
    max_length: int = 1024
    fold_idx: int = 0
    optim_type: str = 'adamw_8bit'
    per_device_train_batch_size: int = 2
    gradient_accumulation_steps: int = 2
    per_device_eval_batch_size: int = 8
    n_epochs: int = 1
    freeze_layers: int = 8
    lr: float = 2e-4
    warmup_steps: int = 20
    lora_r: int = 32
    lora_alpha: float = lora_r * 2
    lora_dropout: float = 0.05
    lora_bias: str = 'none'
    
config = Config()

In [4]:
config = Config()

training_args = TrainingArguments(
    output_dir='output',
    overwrite_output_dir=True,
    report_to='none',
    num_train_epochs=config.n_epochs,
    per_device_train_batch_size=config.per_device_train_batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    per_device_eval_batch_size=config.per_device_eval_batch_size,
    logging_steps=10,
    eval_strategy='epoch',
    save_strategy='steps',
    save_steps=50,
    optim=config.optim_type,
    fp16=True,
    learning_rate=config.lr,
    warmup_steps=config.warmup_steps,
)

In [5]:
lora_config = LoraConfig(
    r=config.lora_r,
    lora_alpha=config.lora_alpha,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'],
    layers_to_transform=[i for i in range(42) if i >= config.freeze_layers],
    lora_dropout=config.lora_dropout,
    bias=config.lora_bias,
    task_type=TaskType.SEQ_CLS,
    use_dora=True
)

In [6]:
tokenizer = GemmaTokenizerFast.from_pretrained(config.checkpoint)
tokenizer.add_eos_token = True
tokenizer.add_sep_token = True
tokenizer.padding_side = 'right'

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [7]:
tr = pd.read_csv('/kaggle/input/dls-data/train.csv.csv')

In [8]:
model = Gemma2ForSequenceClassification.from_pretrained(
    config.checkpoint,
    num_labels=50,
    torch_dtype=torch.float16,
    device_map="auto"
)

model.config.use_cache = False
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.config.pad_token_id = tokenizer.pad_token_id
model

config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/6.13G [00:00<?, ?B/s]

Some weights of Gemma2ForSequenceClassification were not initialized from the model checkpoint at unsloth/gemma-2-9b-it-bnb-4bit and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): Gemma2ForSequenceClassification(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 3584, padding_idx=0)
        (layers): ModuleList(
          (0-7): 8 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): Linear4bit(in_features=3584, out_features=4096, bias=False)
              (k_proj): Linear4bit(in_features=3584, out_features=2048, bias=False)
              (v_proj): Linear4bit(in_features=3584, out_features=2048, bias=False)
              (o_proj): Linear4bit(in_features=4096, out_features=3584, bias=False)
              (rotary_emb): Gemma2RotaryEmbedding()
            )
            (mlp): Gemma2MLP(
              (gate_proj): Linear4bit(in_features=3584, out_features=14336, bias=False)
              (up_proj): Linear4bit(in_features=3584, out_features=14336, bias=False)
              (down_proj): Linear4bit(in_features=14336, out_features=3584, bias=

In [9]:
model.print_trainable_parameters()

trainable params: 89,134,080 || all params: 9,331,019,264 || trainable%: 0.9552


In [10]:
train = pd.read_csv('/kaggle/input/dls-data/train.csv.csv').drop(columns=['Unnamed: 0'])

In [11]:
train['tags'].fillna('{PASS}', inplace=True)
train['tags'] = train['tags'].apply(lambda row: row[1:-1])
train['assessment'] = train['assessment'].astype(str)
train['total'] = train['assessment'].str.cat(train['tags'], sep=' <sep> ')
train['total'] = train['total'].str.cat(train['text'], sep=' <sep> ')

/tmp/ipykernel_24/625557437.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['tags'].fillna('{PASS}', inplace=True)


In [12]:
columns_to_vectorize = [f'trend_id_res{i}' for i in range(50)]
train['target'] = train[columns_to_vectorize].apply(lambda row: list(row), axis=1)

In [13]:
train.drop(
    columns=[
        'index',
        'assessment',
        'text',
        'tags'
    ],
    axis=1,
    inplace=True
)

train.drop(
    columns=columns_to_vectorize,
    axis=1,
    inplace=True
)

In [14]:
train

,total,target
0,"6.0 <sep> ASSORTMENT,PROMOTIONS,DELIVERY <sep>...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"4.0 <sep> ASSORTMENT,PRICE,PRODUCTS_QUALITY,DE...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"6.0 <sep> DELIVERY,PROMOTIONS,PRICE,ASSORTMENT...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"6.0 <sep> PRICE,PROMOTIONS,ASSORTMENT <sep> На...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,"6.0 <sep> PRICE,PROMOTIONS <sep> Можно немного...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
...,...,...
4618,2.0 <sep> PRODUCTS_QUALITY <sep> Привезли прок...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4619,6.0 <sep> DELIVERY <sep> пойдет,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4620,"6.0 <sep> PROMOTIONS,PRICE,SUPPORT,PRODUCTS_QU...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4621,"2.0 <sep> DELIVERY,PRODUCTS_QUALITY <sep> Очен...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [15]:
idx = train.index.tolist()
text = train.total.tolist()
labels = train.target.tolist()

In [16]:
labels = np.array(labels, dtype=int)

In [17]:
label_weights = 1 - labels.sum(axis=0) / labels.sum()

In [18]:
row_ids = np.arange(len(labels))
train_idx, y_train, val_idx, y_val = iterative_train_test_split(row_ids[:,np.newaxis], labels, test_size=0.1)
x_train = [text[i] for i in train_idx.flatten()]
x_val = [text[i] for i in val_idx.flatten()]

In [19]:
ds = DatasetDict({
    'train': Dataset.from_dict({'text': [str(x) for x in x_train], 'labels': y_train}),
    'val': Dataset.from_dict({'text': [str(x) for x in x_val], 'labels': y_val})
})

In [20]:
def tokenize_examples(examples, tokenizer):
    tokenized_inputs = tokenizer(examples['text'], max_length=config.max_length, truncation=True)
    tokenized_inputs['labels'] = examples['labels']
    return tokenized_inputs

ds = ds.map(functools.partial(tokenize_examples, tokenizer=tokenizer), batched=True)
ds = ds.with_format('torch')

Map:   0%|          | 0/4144 [00:00<?, ? examples/s]

Map:   0%|          | 0/479 [00:00<?, ? examples/s]

In [21]:
def collate_fn(batch, tokenizer):
    dict_keys = ['input_ids', 'attention_mask', 'labels']
    d = {k: [dic[k] for dic in batch] for k in dict_keys}
    d['input_ids'] = torch.nn.utils.rnn.pad_sequence(
        d['input_ids'], batch_first=True, padding_value=tokenizer.pad_token_id
    )
    d['attention_mask'] = torch.nn.utils.rnn.pad_sequence(
        d['attention_mask'], batch_first=True, padding_value=0
    )
    d['labels'] = torch.stack(d['labels'])
    return d

In [22]:
def compute_metrics(p):
    predictions, labels = p
    f1_micro = f1_score(labels, predictions > 0, average='micro')
    f1_macro = f1_score(labels, predictions > 0, average='macro')
    f1_weighted = f1_score(labels, predictions > 0, average='weighted')
    accuracy = accuracy_score(labels, predictions > 0)
    
    return {
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted,
        'accuracy': accuracy
    }

In [23]:
class CustomTrainer(Trainer):
    def __init__(self, label_weights, **kwargs):
        super().__init__(**kwargs)
        self.label_weights = label_weights
    
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop('labels')
        
        outputs = model(**inputs)
        logits = outputs.get('logits')
        
        loss = F.binary_cross_entropy_with_logits(logits, labels.to(torch.float32), pos_weight=self.label_weights)
        return (loss, outputs) if return_outputs else loss

In [24]:
trainer = CustomTrainer(
    args=training_args, 
    model=model,
    tokenizer=tokenizer,
    train_dataset=ds['train'],
    eval_dataset=ds['val'],
    data_collator=functools.partial(collate_fn, tokenizer=tokenizer),
    compute_metrics=compute_metrics,
    label_weights=torch.tensor(label_weights, device=model.device)
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_aut

Epoch,Training Loss,Validation Loss,F1 Micro,F1 Macro,F1 Weighted,Accuracy
1,0.040600,0.034732,0.748006,0.587604,0.729693,0.551148


/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an except

TrainOutput(global_step=1036, training_loss=0.06678777311759566, metrics={'train_runtime': 12941.9711, 'train_samples_per_second': 0.32, 'train_steps_per_second': 0.08, 'total_flos': 1.0789424666431488e+16, 'train_loss': 0.06678777311759566, 'epoch': 1.0})